# 📌 Résumé du projet
Dans le cadre de l’appel à manifestation d’intérêt de l’ADEME, notre équipe CesiCDP développe une solution intelligente visant à optimiser les tournées de livraison de biens ou services dans un environnement urbain complexe. L’objectif est de réduire les déplacements et la consommation énergétique tout en prenant en compte des contraintes réalistes et dynamiques du terrain : routes fermées, ralenties, ou évoluant dans le temps.

Nous modélisons ce problème sous forme d’un graphe pondéré représentant un réseau routier. Notre approche repose sur une méthode approchée, capable de s’adapter à des situations dynamiques et incomplètes, et d’obtenir de bonnes solutions rapidement sans garantie d’optimalité.

# 🧭 Introduction
Dans ce notebook, nous analysons et testons l'efficacité de notre algorithme de Recuit Simulé (Simulated Annealing) appliqué à l'optimisation des tournées de livraison.

L'objectif est de :
- Évaluer les performances de l'algorithme en termes de qualité des solutions et de temps d'exécution.
- Étudier la complexité algorithmique et spatiale.
- Comparer les résultats obtenus avec d'autres approches.

# ⚠️ Contraintes
Et les contraintes sont : 
- Routes dynamiques ou perturbations : Simuler des changements dynamiques dans les coûts ou la disponibilité des routes pendant la résolution.
- Utilisation de plusieurs véhicules: Il peut y avoir plusieurs sous-tournées plutôt qu'une seule grande.
- Coût ou restriction de passage sur certaines arêtes : Certaines routes peuvent être plus coûteuses ou interdites (par exemple, travaux ou routes bloquées).
- Ratio d'embouteillage moyen : Les données de Bison Futé sont utilisées pour calculer un ratio d'embouteillage moyen en fonction de l'heure.
- Capacités du véhicule : Chaque véhicule a une capacité limite pour transporter des marchandises ou des passagers.


# 🧱 Structure du graphe
Chaque ville est un nœud, chaque route une arête pondérée. Le graphe peut être enrichi par des données géographiques.
```python
from graph import Graph
graph = Graph.load('./data/datasets/size_200/graph_size200_density0.01.pkl')
graph.plot()
```


# 🔄 Pseudo-code de l'algorithme de Recuit Simulé
FONCTION SimulatedAnnealing(graph, initial_temp, min_temp, cooling_rate, max_iterations, num_vehicles):
Entrée :
    - graph : graphe contenant les nœuds et arêtes pondérées
    - initial_temp : température initiale
    - min_temp : température minimale
    - cooling_rate ∈ (0,1) : facteur de refroidissement
    - max_iterations : nombre maximal d’itérations à température constante
    - num_vehicles : nombre de véhicules disponibles

Fonctions :
    - initialize_solution(nodes, start_node, num_vehicles, graph) :
        génère une solution initiale valide répartie sur plusieurs véhicules
    - compute_total_cost(graph, solution) :
        retourne le coût total de la solution (somme des distances pour tous les véhicules)
    - generate_neighbor_multi_vehicle(graph, solution) :
        génère une solution voisine (modification locale d’un ou plusieurs trajets)
    - validate_solution(graph, solution) :
        vérifie que la solution est valide (tous les sommets sont visités une fois, chemins valides)
    - save_graph_png(path, show_labels, solution) :
        enregistre une visualisation graphique de la solution actuelle

Algorithme :

Début :

    temps_début ← temps_actuel                                     // début du chronomètre
    T ← initial_temp                           // température initiale

    nodes ← liste des nœuds du graphe
    start_node ← nœud de départ aléatoire
    retirer start_node de nodes

    current_solution ← initialize_solution(nodes, start_node, num_vehicles, graph)
    best_solution ← copie de current_solution
    current_cost ← compute_total_cost(graph, current_solution)
    best_cost ← current_cost

    Afficher "Initial solution cost"

    Pour chaque véhicule dans best_solution :
        enregistrer le chemin dans le graphe

    number_iterations ← 0
    number_saves ← 0

    Tant que T > min_temp :

        Pour i de 1 à max_iterations :
            neighbor ← generate_neighbor_multi_vehicle(graph, current_solution)
            cost_neighbor ← compute_total_cost(graph, neighbor)
            Δ ← cost_neighbor - current_cost

            Si Δ < 0  :
                //Amélioration → accepter directement
                current_solution ← neighbor
                current_cost ← cost_neighbor

                Si cost_neighbor < best_cost :
                    best_solution ← copie de neighbor
                    best_cost ← cost_neighbor
                    Pour chaque véhicule :
                        enregistrer le chemin dans le graphe
            Sinon :
                //Moins bon → accepter avec probabilité exp(-Δ / T)
                p ← exp(-Δ / T)
                Si random(0,1) < p :
                    current_solution ← neighbor
                    current_cost ← cost_neighbor

            Si number_iterations mod 500 = 0 :
                enregistrer la solution actuelle en image
                incrémenter number_saves

            incrémenter number_iterations

        Afficher T, current_cost, best_cost
        T ← T × cooling_rate

    Répéter 10 fois :
        enregistrer best_solution pour figer dans l’animation finale

    Si validate_solution(graph, best_solution) = Faux :
        Lever une erreur : solution invalide
    
    temps_écoulé = temps_actuel - temps_début

    Afficher "Final solution cost"
    Afficher "Elapsed time"

    Retourner best_solution, temps_écoulé
![Flowchart](flowchart_simulated_annealing.svg)


# 🧮 Analyse de la complexité spatiale
- Espace requis : O(n + m + k.n) où :
  - n : nombre de villes
  - m : nombre d'arêtes
  - k : nombre de véhicules

Dans notre cas, la classe personnalisée "Graph" stocke le graph en lui même grâce à Networkx. Ainsi qu'un dictionnaire qui prend pour clé l'id du véhicule et en valeur le chemin qu'il emprunte.

Ainsi, le stockage du graph en lui même prend une taille de O(n + m) car il est stocké dans un dictionnaire avec pour clé les sommets et pour valeurs un deuxième dictionnaire, qui lui stock en clé un sommet auquel le premier sommet est lié (créant de ce fait une arrête) et en valeur un dernier dictionnaire qui stock les différents attributs de chaque arrêtes tel que le poids de ces dernières.

Dans le pire des cas on a donc besoins de stocker n sommet, et m arrêtes.

En ce qui concerne les véhicules et leur trajet, cela demande une taille de O(k.n) car au pire des cas chaque véhicule passera par les n sommets. 
Ce faisant nous aurions k trajets de n sommets

Ci-dessous un exemple de stockage d'un graph en utilisant Networkx : 

In [15]:
{
    'A': {                      #Clé = Sommet : Valeur = Dictionnaire des arrêtes
        'B': {'weight': 5},     #Clé = Sommet à la fin de l'arrête : Valeur = Dictionnaire des attributs de l'arrêtes
        'C': {'weight': 3}
    },
    'B': {
        'A': {'weight': 5},
        'D': {'weight': 2}
    },
    'C': {
        'A': {'weight': 3}
    },
    'D': {
        'B': {'weight': 2}
    }
}

{'A': {'B': {'weight': 5}, 'C': {'weight': 3}},
 'B': {'A': {'weight': 5}, 'D': {'weight': 2}},
 'C': {'A': {'weight': 3}},
 'D': {'B': {'weight': 2}}}

Ci-dessous un graph montrant l'utilisation réel de mémoire pour des graphs de tailles et densitées variables :

![Graph de la complexité spatiale](../data/test/spatial_complexity.png)

On voit avec le graphique ci dessous que le nombre en sommet influe peux la taille prise par les graphiques créer pour des densitées faible, mais plus la densité augmente et plus ces derniers ont un impact. 

Étant donnée que la densité donne une proportion de liaison entre tous les sommets, il n'est donc pas surprenant de voir qu'avec une proportion de liaisons plus grande entre les sommets, le stockage du graph dans son ensemble prend beaucoup plus de place, surtout avec un nombre important de sommet.

C'est ce qui explique pourquoi lorsque l'on est entre 100 et 300 sommets, les lignes sont plutôt proches, du fait que l'influence du pourcentage est moindre. 
Là où à partir des 400-500 sommets et ce de manière de plus en plus notable en augmentant le nombre des sommets, les lignes se sépare de plus en plus.

In [2]:
import sys
from pathlib import Path
import tracemalloc
import pandas as pd
import numpy as np
from time import time
import matplotlib.pyplot as plt
import seaborn as sns

notebooks_dir = Path().resolve()
src_dir = notebooks_dir.parent / "src"
sys.path.append(str(src_dir))

from graph import Graph


def test_spatial_complexity():
    results = []
    tailles = [i for i in range(100, 1000, 100)]
    densites = [0.001, 0.01, 0.05, 0.1, 0.15, 0.20, 0.25, 0.3]

    for n in tailles:
        for d in densites:
            g = Graph()
            print(f"Generating graph for n vertices={n}, density={d}")

            tracemalloc.start()
            g.generate_geo_graph(n=n, density=d)
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            results.append({
                'size': n,
                'density': d,
                'used_memory_MB': current / 1024**2,
            })

            print("Success")

    
    results_df = pd.DataFrame(results)

    # === Affichage console pour CI ===
    print("\n=== Résultats de la mesure de complexité spatiale ===\n")
    print(results_df.to_string(index=False))

    # Créer le dossier si nécessaire
    output_dir = Path(__file__).resolve().parent.parent / "data" / "test"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Créer le graphique
    plt.figure(figsize=(10, 6))
    sns.lineplot(
        data=results_df,
        x='size',
        y='used_memory_MB',
        hue='density',
        marker='o',
        palette='viridis'
    )
    plt.title("Memory used according to the graph size")
    plt.xlabel("Number of vertices (n)")
    plt.ylabel("Memory used (MB)")
    plt.grid(True)
    plt.tight_layout()

    # Enregistrer le fichier image
    plot_path = output_dir / "spatial_complexity.png"
    plt.savefig(plot_path)
    print(f"Plot saved to {plot_path}")


print("Starting space complexity test")
test_spatial_complexity()




Starting space complexity test
Generating graph for n vertices=100, density=0.001


FileNotFoundError: [Errno 2] No such file or directory: './data/FR/cities_of_france.txt'

# ⚙️ Choix des paramètres
Utiliser librairie octuna

# 🧪 Test de performance
Grâce à la librairie cProfile, nous pouvons tracer les performances de l'algorithme de manière précise, et la fonction time.perf_counter() elle nous permet de récupérer des informations sur le temps d'exécution de l'algorithme, tout ceci sur une nombre variable de sommet et de densité.

Ainsi pour le temps on obtient le graph suivant : 

![Graph des tests de performances](../data/test/performance_profiling.png)


On obtient les valeurs grâce à time.perf_counter() en début et en fin d'algorithme, faire la différences des deux appels donne une mesure précise du temps total d'exécution.

En ce qui concerne les performances on obtient un fichier texte grâce à cProfile qui rassemble les données comme ci-dessous :   

In [4]:
profiling_output = """
# appels   temps propre (s)     /appel     temps cumulé (s)      /appel cumulé     fonction
1          0.070                0.070      15.625                15.625            simulated_annealing
27600      0.228                0.000      8.178                 0.000             generate_neighbor_multi_vehicle
"""

print(profiling_output)


# appels   temps propre (s)     /appel     temps cumulé (s)      /appel cumulé     fonction
1          0.070                0.070      15.625                15.625            simulated_annealing
27600      0.228                0.000      8.178                 0.000             generate_neighbor_multi_vehicle



On retrouve alors le nombre de fois où la fonction est appelée, le temps pris par la fonction en elle même, le temps pris par appel de la fonction, le temps cumulé d'exécution de la fonction et enfin le temps cumulé par appel de la fonction. Tout cela suivi par le nom de la fonction en elle même pour que l'on sache quelle fonction est concernée. Ceci nous permet au final de traquer de manière précise ce qui se passe lors de l'exécution de l'algorithme.

# ⚖️ Comparaison avec d'autres algorithmes
Comparaison avec une heuristique gloutonne sur les mêmes instances :
```python
from compare import compare_algorithms
compare_algorithms(graph)
```


# 📽️ Rendu visuel avec GIF
```python
from visual import render_solution_gif
render_solution_gif(best_solution, './animations/solution.gif')
```
![GIF](./animations/solution.gif)


# ✅ Conclusion
- Le recuit simulé est efficace sur les graphes denses.
- Robuste face aux perturbations.
- Temps raisonnables et résultats compétitifs.
- Possibilités d'amélioration avec des hybrides heuristiques.
